In [1]:
#Dependencies
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser
import re
import pandas as pd
import time

In [2]:
#Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
#Define database and collection
db = client.mars_db

In [4]:
#Setup for splinter
executable_path = {'executable_path': '../For Submission/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [5]:
#Website to scrape with BeautifulSoup
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1)
html = browser.html
soup = bs(html, 'html.parser')

In [6]:
#Pull info needed from page as list
results_text = soup.find_all('div',class_="content_title")
news_title = results_text[1].text
print(news_title)

NASA's Mars Rover Drivers Need Your Help


In [7]:
#Pull info needed from page as list
para_text = soup.find('div',class_="article_teaser_body")
news_p = para_text.text
print(news_p)

Using an online tool to label Martian terrain types, you can train an artificial intelligence algorithm that could improve the way engineers guide the Curiosity rover.


In [8]:
#Website to scrape with BeautifulSoup
jpl_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(jpl_url)
jpl_html = browser.html
jpl_soup = bs(jpl_html, 'html.parser')

In [9]:
#Pull image url
image_result = jpl_soup.find('article')['style'].replace("background-image: url('",'').replace("');",'')

image_url= "https://www.jpl.nasa.gov" + image_result

print(image_url)

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA17896-1920x1200.jpg


In [10]:
#Website to scrape with BeautifulSoup
twitter_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(twitter_url)
time.sleep(5)
twitter_html = browser.html
twitter_soup = bs(twitter_html, 'html.parser')

In [11]:
twitter_results = twitter_soup.find_all(text=re.compile("InSight"))
mars_weather = twitter_results[0]
print(mars_weather)

InSight sol 548 (2020-06-11) low -91.2ºC (-132.1ºF) high -2.8ºC (26.9ºF)
winds from the SW at 4.8 m/s (10.8 mph) gusting to 20.1 m/s (45.0 mph)
pressure at 7.40 hPa


In [12]:
#Website to scrape with BeautifulSoup
facts_url = 'https://space-facts.com/mars/'
facts_table = pd.read_html(facts_url)

In [13]:
facts_df = facts_table[0]
print(facts_df)

                      0                              1
0  Equatorial Diameter:                       6,792 km
1       Polar Diameter:                       6,752 km
2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
3                Moons:            2 (Phobos & Deimos)
4       Orbit Distance:       227,943,824 km (1.38 AU)
5         Orbit Period:           687 days (1.9 years)
6  Surface Temperature:                   -87 to -5 °C
7         First Record:              2nd millennium BC
8          Recorded By:           Egyptian astronomers


In [14]:
html_table = facts_df.to_html()
print(html_table)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>0</th>
      <th>1</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Moons:</td>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Orbit Distance:</td>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Orbit Period:</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>6</th>
      <td>Surface Temperature:</td>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>7</th>
      <td>First Record:</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>8</th>
      <td>Recorded

In [15]:
#Website to scrape with BeautifulSoup
astrog_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(astrog_url)
astrog_html = browser.html
astrog_soup = bs(astrog_html, 'html.parser')

In [16]:
#Parse results and scrape titles and img links
astrog_results = astrog_soup.find_all('div',class_="description")
hemisphere_image_urls = []
for result in astrog_results:
    
    #Add title to dictionary
    title = result.h3.text.replace(" Enhanced","")
    
    #Add url for full size image to dictionary
    browser.visit("https://astrogeology.usgs.gov/" + result.a['href'])
    nested_html = browser.html
    nested_soup = bs(nested_html, 'html.parser')
    nested_results = nested_soup.find('img',class_="wide-image")
    link = "https://astrogeology.usgs.gov/" + nested_results['src']
    
    #Add dictionary to list
    hemisphere_image_urls.append({"title":title,"img_url":link})

print(hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere', 'img_url': 'https://astrogeology.usgs.gov//cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere', 'img_url': 'https://astrogeology.usgs.gov//cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere', 'img_url': 'https://astrogeology.usgs.gov//cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere', 'img_url': 'https://astrogeology.usgs.gov//cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]


In [17]:
# Create dictionary with scraped data
final_dict={"news_title": news_title,
          "news_p": news_p,
          "featured_img_url": image_url,
          "mars_weather": mars_weather,
          "hemisphere_img_urls": hemisphere_image_urls,
          "html_table":html_table}
print(final_dict)

{'news_title': "NASA's Mars Rover Drivers Need Your Help", 'news_p': 'Using an online tool to label Martian terrain types, you can train an artificial intelligence algorithm that could improve the way engineers guide the Curiosity rover.', 'featured_img_url': 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA17896-1920x1200.jpg', 'mars_weather': 'InSight sol 548 (2020-06-11) low -91.2ºC (-132.1ºF) high -2.8ºC (26.9ºF)\nwinds from the SW at 4.8 m/s (10.8 mph) gusting to 20.1 m/s (45.0 mph)\npressure at 7.40 hPa', 'hemisphere_img_urls': [{'title': 'Cerberus Hemisphere', 'img_url': 'https://astrogeology.usgs.gov//cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere', 'img_url': 'https://astrogeology.usgs.gov//cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere', 'img_url': 'https://astrogeology.usgs.gov//cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrt

In [18]:
db.mars.insert_one(final_dict)